In [1]:
import pandas as pd
import numpy as np

In [2]:
%run load_mnist_dataset.py

In [3]:
## Script create_dense_model.py
import tensorflow as tf
import numpy as np
from tensorflow import keras
DIGIT_CLASSES = 10
RESHAPED = 28 * 28 ## 784 pixels per image
import time
from datetime import datetime


def create_dense_model(hidden_layers=1, hidden_units_per_layer=128):
    """
    create a keras sequential model with the specified
    number of hidden_layers and number of units per layer
    """
    DIGIT_CLASSES = 10
    RESHAPED = 28 * 28 ## 784 pixels per image
    model = tf.keras.models.Sequential()
    count = 0
    
    
    count += 1
    model.add(keras.layers.Dense(units=hidden_units_per_layer,
        input_shape=(RESHAPED,),
        name=f'dense_layer_{count}', activation='relu'))
    
    for i in range(1, hidden_layers):
        count += 1
        model.add(keras.layers.Dense(units=hidden_units_per_layer,
           name=f'dense_layer_{count}', activation='relu'))
        
    count += 1        
    model.add(keras.layers.Dense(DIGIT_CLASSES,
       name=f'dense_layer_{count}', activation='softmax'))

    return model


def run_dense_model(model_name=None, model_fn=None,  train_size=50000, optimizer='Adam', verbose=0):
    """
    Create and run a dense Keras sequential model over the MNIST training data,
    using the specified numbers of hidden_layers, hidden_units_per_layer, and optimizer.
    Maybe add optional dropout TBD,
    Return values:
    history, training_time, test_accuracy, model
    """
    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss',verbose=1,patience=3,min_delta=0.0001)
    
    BATCH_SIZE = 128
    EPOCHS = 400
    
    ## Load the training, validation, and test data
    X_train, X_valid, X_test, Y_train, Y_valid, Y_test = load_mnist_dataset(train_size=train_size, flatten=True)
    
    model = model_fn()
    
    start_time = time.time()

    # compiling the model
    model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    #training the model
    history = model.fit(X_train, Y_train,
                        batch_size=BATCH_SIZE, epochs=EPOCHS,
                        verbose=verbose,  validation_data=(X_valid,Y_valid),
                        callbacks=[early_stopping]
                       )
    
    elapsed_time = time.time() - start_time
    
    test_loss, test_accuracy = model.evaluate(X_test, Y_test)
    print(f'training elapsed_time = {elapsed_time:10.1f}')
    print('Test accuracy:', test_accuracy)
    
    return [model_name, train_size, elapsed_time, test_accuracy]

In [4]:
## The most accurate Dense model from earlier testing appeared to be
## the model with 1 hidden layer of 192 unit, and using the Adam Optimizer, but no Dropout.

## We will re-create that model here, and train it with successively smaller training sets,
## in order to compare it with the CNN models on the same data sets.

## model5 = create_dense_model(hidden_layers=1, hidden_units_per_layer=192)

def create_Model5_Dense_1_Hidden_Layer():
    model = create_dense_model(hidden_layers=1, hidden_units_per_layer=192)
    return model

model5 = create_Model5_Dense_1_Hidden_Layer()
model5.summary()
model5 = create_Model5_Dense_1_Hidden_Layer


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 192)               150720    
                                                                 
 dense_layer_2 (Dense)       (None, 10)                1930      
                                                                 
Total params: 152,650
Trainable params: 152,650
Non-trainable params: 0
_________________________________________________________________


In [5]:
## Create a model_set, to reuse existing code to train and evaluate the dense model
## over a range of training sample sizes.
model_set = [
             ('model5',model5),
            ]

train_sizes = [50000, 20000, 10000, 5000, 2500, 1200, 600, 450, 300]

def run_model_set_size(train_sizes=[50000], model_set=None, repeat=1):
    now = datetime.now()
    result_name = now.strftime("data/CNN_DATA_%Y-%m-%d_%H_%M.csv")
    results = []
    for train_size in train_sizes:
        for i in range(repeat):
            for model_name, model in model_set:
                result = run_dense_model(model_name=model_name, model_fn=model, train_size=train_size)
                results.append(result)
            df = pd.DataFrame(results)
            df.columns=['model_name','train_size','training_time','test_accuracy']
            display(df)
            print(f'Saving results to file {result_name}')
            df.to_csv(result_name,index=False)
    return results

results = run_model_set_size(train_sizes, model_set = model_set, repeat=4)

50000 train samples
10000 validation samples
10000 test samples
Epoch 10: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.0824 - accuracy: 0.9773
training elapsed_time =       14.2
Test accuracy: 0.9772999882698059


,model_name,train_size,training_time,test_accuracy
0,model5,50000,14.21803,0.9773


Saving results to file data/CNN_DATA_2022-10-01_17_56.csv
50000 train samples
10000 validation samples
10000 test samples
Epoch 14: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.0769 - accuracy: 0.9781
training elapsed_time =       20.6
Test accuracy: 0.9781000018119812


,model_name,train_size,training_time,test_accuracy
0,model5,50000,14.218030,0.9773
1,model5,50000,20.612055,0.9781


Saving results to file data/CNN_DATA_2022-10-01_17_56.csv
50000 train samples
10000 validation samples
10000 test samples
Epoch 16: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.0745 - accuracy: 0.9787
training elapsed_time =       22.8
Test accuracy: 0.9786999821662903


...

Saving results to file data/CNN_DATA_2022-10-01_17_56.csv
300 train samples
59700 validation samples
10000 test samples
Epoch 22: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.6848 - accuracy: 0.7965
training elapsed_time =       28.3
Test accuracy: 0.796500027179718


,model_name,train_size,training_time,test_accuracy
0,model5,50000,14.218030,0.9773
1,model5,50000,20.612055,0.9781
2,model5,50000,22.752473,0.9787
3,model5,50000,15.953516,0.9763
4,model5,20000,23.176431,0.9685
5,model5,20000,19.400879,0.9651
6,model5,20000,16.895140,0.9694
7,model5,20000,22.754974,0.9700
8,model5,10000,23.394137,0.9534
9,model5,10000,26.489734,0.9537


Saving results to file data/CNN_DATA_2022-10-01_17_56.csv
300 train samples
59700 validation samples
10000 test samples
Epoch 27: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.6516 - accuracy: 0.8002
training elapsed_time =       34.7
Test accuracy: 0.8001999855041504


,model_name,train_size,training_time,test_accuracy
0,model5,50000,14.218030,0.9773
1,model5,50000,20.612055,0.9781
2,model5,50000,22.752473,0.9787
3,model5,50000,15.953516,0.9763
4,model5,20000,23.176431,0.9685
5,model5,20000,19.400879,0.9651
6,model5,20000,16.895140,0.9694
7,model5,20000,22.754974,0.9700
8,model5,10000,23.394137,0.9534
9,model5,10000,26.489734,0.9537


Saving results to file data/CNN_DATA_2022-10-01_17_56.csv


In [8]:
import pandas as pd
df1 = pd.read_csv('data/CNN_COMPARE_TRAINING_SET_SIZE.csv')
df2 = pd.read_csv('data/CNN_DATA_2022-10-01_17_56.csv')
df = pd.concat([df1,df2])
df.drop(df.columns[0],axis=1,inplace=True)
df

,model_name,train_size,training_time,test_accuracy
0,model1,50000,23.973921,0.9917
1,model2,50000,49.130800,0.9893
2,model3,50000,46.421665,0.9917
3,model4,50000,52.340361,0.9922
4,model1,50000,24.637258,0.9903
...,...,...,...,...
31,model5,450,28.312823,0.8255
32,model5,300,28.155860,0.7940
33,model5,300,23.430303,0.7860
34,model5,300,28.318739,0.7965


In [25]:
import numpy as np
grouped = df.groupby(['model_name','train_size'])
averages = grouped[['training_time','test_accuracy']].aggregate(np.mean)
averages.reset_index(inplace=True)
averages

,model_name,train_size,training_time,test_accuracy
0,model1,300,32.856159,0.852675
1,model1,450,29.092179,0.873050
2,model1,600,37.484657,0.920750
3,model1,1200,25.634220,0.935075
4,model1,2500,28.482949,0.966700
5,model1,5000,25.949396,0.976375
6,model1,10000,21.590354,0.982175
7,model1,20000,24.930123,0.986750
8,model1,50000,27.671751,0.990900
9,model2,300,33.456769,0.815675


In [26]:
averages.drop(averages.columns[2],axis=1,inplace=True)
averages.to_csv('data/MEAN_ACCURACY_BY_MODEL_AND_TRAINING_SIZE.csv',index=False)
averages

,model_name,train_size,test_accuracy
0,model1,300,0.852675
1,model1,450,0.873050
2,model1,600,0.920750
3,model1,1200,0.935075
4,model1,2500,0.966700
5,model1,5000,0.976375
6,model1,10000,0.982175
7,model1,20000,0.986750
8,model1,50000,0.990900
9,model2,300,0.815675
